In [1]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
def saveFile(model, filename):
    pickle.dump(model, open(filename, 'wb'))
    
def loadFile(filename):
    return pickle.load(open(filename, 'rb'))

In [3]:
backward = loadFile("./Backwards Regression/BackwardRegressionMSE")

In [4]:
forward = loadFile("./Forward Selection/ForwardRegressionMSE")

In [5]:
forward = forward[backward.columns.values]

In [6]:
backward.head()

model selection  \
Folds item_nbr                                                                
fold5 1         <statsmodels.regression.linear_model.Regressio...  backward   
fold1 2         <statsmodels.regression.linear_model.Regressio...  backward   
fold2 3         <statsmodels.regression.linear_model.Regressio...  backward   
fold1 4         <statsmodels.regression.linear_model.Regressio...  backward   
      5         <statsmodels.regression.linear_model.Regressio...  backward   

                rsquared_adj          MSE  
Folds item_nbr                             
fold5 1             0.048489     0.130139  
fold1 2             0.061097     0.918617  
fold2 3             0.089020     0.075939  
fold1 4             0.007371     0.026946  
      5             0.358673  5964.703593

In [7]:
forward.head()

model selection  \
Folds item_nbr                                                                
fold5 1         <statsmodels.regression.linear_model.Regressio...   forward   
fold1 2         <statsmodels.regression.linear_model.Regressio...   forward   
fold2 3         <statsmodels.regression.linear_model.Regressio...   forward   
fold1 4         <statsmodels.regression.linear_model.Regressio...   forward   
      5         <statsmodels.regression.linear_model.Regressio...   forward   

                rsquared_adj          MSE  
Folds item_nbr                             
fold5 1             0.004529     0.130139  
fold1 2             0.001559     0.987207  
fold2 3             0.002397     0.078389  
fold1 4             0.001655     0.026946  
      5             0.176935  3611.746326

In [8]:
forward.set_index(forward.index.droplevel(0),inplace=True)
backward.set_index(backward.index.droplevel(0),inplace=True)

In [11]:
backward.shape

(110, 4)

In [12]:
forward.shape

(110, 4)

In [13]:
itemModels = backward.append(forward)

In [14]:
itemModels.shape

(220, 4)

In [15]:
itemModels.head()

,model,selection,rsquared_adj,MSE
item_nbr,,,,
1,<statsmodels.regression.linear_model.Regressio...,backward,0.048489,0.130139
2,<statsmodels.regression.linear_model.Regressio...,backward,0.061097,0.918617
3,<statsmodels.regression.linear_model.Regressio...,backward,0.089020,0.075939
4,<statsmodels.regression.linear_model.Regressio...,backward,0.007371,0.026946
5,<statsmodels.regression.linear_model.Regressio...,backward,0.358673,5964.703593


In [16]:
itemModels.set_index([itemModels.index,"selection"],inplace = True)

In [17]:
itemModels.xs(1,axis=0,drop_level=False)

model  \
item_nbr selection                                                      
1        backward   <statsmodels.regression.linear_model.Regressio...   
         forward    <statsmodels.regression.linear_model.Regressio...   

                    rsquared_adj       MSE  
item_nbr selection                          
1        backward       0.048489  0.130139  
         forward        0.004529  0.130139

In [18]:
itemModels.loc[(1,"backward"),"MSE"] - itemModels.loc[(1,"forward"),"MSE"]

0.0

In [19]:
optModels = itemModels.groupby("item_nbr")["MSE"].idxmin().values
itemOptModels = itemModels.loc[optModels]
itemOptModels = itemOptModels.reset_index().set_index("item_nbr")
itemOptModels = itemOptModels[forward.columns.values]

In [20]:
itemOptModels.shape

(110, 4)

In [21]:
itemOptModels.head()

,model,selection,rsquared_adj,MSE
item_nbr,,,,
1,<statsmodels.regression.linear_model.Regressio...,backward,0.048489,0.130139
2,<statsmodels.regression.linear_model.Regressio...,backward,0.061097,0.918617
3,<statsmodels.regression.linear_model.Regressio...,backward,0.089020,0.075939
4,<statsmodels.regression.linear_model.Regressio...,backward,0.007371,0.026946
5,<statsmodels.regression.linear_model.Regressio...,forward,0.176935,3611.746326


In [22]:
itemOptModels.selection.value_counts()

backward    101
forward       9
Name: selection, dtype: int64

In [23]:
itemOptModels[itemOptModels.selection == "forward"]

,model,selection,rsquared_adj,MSE
item_nbr,,,,
5,<statsmodels.regression.linear_model.Regressio...,forward,0.176935,3611.746326
9,<statsmodels.regression.linear_model.Regressio...,forward,0.124083,3679.865505
15,<statsmodels.regression.linear_model.Regressio...,forward,0.061664,4.537980
37,<statsmodels.regression.linear_model.Regressio...,forward,0.000881,252.797224
41,<statsmodels.regression.linear_model.Regressio...,forward,0.058256,60.123571
45,<statsmodels.regression.linear_model.Regressio...,forward,0.450041,6377.375340
50,<statsmodels.regression.linear_model.Regressio...,forward,0.031249,0.793357
51,<statsmodels.regression.linear_model.Regressio...,forward,0.054724,0.603157
93,<statsmodels.regression.linear_model.Regressio...,forward,0.059316,32.509799


In [24]:
deltaModels = backward.MSE - forward.MSE

In [25]:
deltaModels[deltaModels == 0].head()

item_nbr
1     0.0
4     0.0
7     0.0
10    0.0
11    0.0
Name: MSE, dtype: float64

In [26]:
itemOptModels.loc[[45]]

,model,selection,rsquared_adj,MSE
item_nbr,,,,
45,<statsmodels.regression.linear_model.Regressio...,forward,0.450041,6377.37534


In [27]:
itemModels.xs(45,level=0,drop_level=False)

model  \
item_nbr selection                                                      
45       backward   <statsmodels.regression.linear_model.Regressio...   
         forward    <statsmodels.regression.linear_model.Regressio...   

                    rsquared_adj           MSE  
item_nbr selection                              
45       backward       0.559505  13476.438759  
         forward        0.450041   6377.375340

In [28]:
itemOptModels.loc[[1]]

,model,selection,rsquared_adj,MSE
item_nbr,,,,
1,<statsmodels.regression.linear_model.Regressio...,backward,0.048489,0.130139


In [29]:
itemModels.xs(1,level=0,drop_level=False)

model  \
item_nbr selection                                                      
1        backward   <statsmodels.regression.linear_model.Regressio...   
         forward    <statsmodels.regression.linear_model.Regressio...   

                    rsquared_adj       MSE  
item_nbr selection                          
1        backward       0.048489  0.130139  
         forward        0.004529  0.130139

In [30]:
itemOptModels.loc[[16]]

,model,selection,rsquared_adj,MSE
item_nbr,,,,
16,<statsmodels.regression.linear_model.Regressio...,backward,0.294095,475.005444


In [31]:
itemModels.xs(16,level=0,drop_level=False)

model  \
item_nbr selection                                                      
16       backward   <statsmodels.regression.linear_model.Regressio...   
         forward    <statsmodels.regression.linear_model.Regressio...   

                    rsquared_adj         MSE  
item_nbr selection                            
16       backward       0.294095  475.005444  
         forward        0.001264  596.721557

In [32]:
saveFile(itemOptModels,"OptRegressionMSE")